In [9]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import io, yaml
import pprint 
pp = pprint.PrettyPrinter(indent=4)


index = 0
experiment = 'DIFF'
folder = 'E:\\Dennis\\Google Drive\\PhD Project\\Experiments\\001-DifferentialDeprivation\\data\\processed\\analysis'
datafile = "{}_{:03d}.csv".format(experiment, index)
metadatafile = "{}_{:03d}.yaml".format(experiment, index)

### DATA INTO DATAFRAME
df = pd.read_csv(os.path.join(folder, datafile), index_col='frame', sep=',')

### METADATA INTO DICT
with open(os.path.join(folder, metadatafile), 'r') as stream:
    try:
        meta = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
pp.pprint(meta)

{   'arena': {   'layout': '6-6 radial',
                 'name': 'topleft',
                 'outer': 260.0,
                 'radius': 213.101847692111,
                 'scale': 8.543,
                 'x': 365.5986,
                 'y': 358.9673},
    'condition': 'SAA',
    'datafile': 'E:/Dennis/Google Drive/PhD '
                'Project/Experiments\\001-DifferentialDeprivation\\data\\processed\\post_tracking\\DIFF_000.csv',
    'datetime': datetime.datetime(2017, 11, 24, 8, 26, 18),
    'fly': {   'genotype': 'Canton Sb',
               'mating': 'mated',
               'metabolic': 'SAA',
               'n_per_arena': 1,
               'sex': 'female'},
    'food_spots': [   {   'r': 1.5,
                          'substr': 'yeast',
                          'x': -9.935580787389284,
                          'y': 0.1723087127862979},
                      {   'r': 1.5,
                          'substr': 'yeast',
                          'x': 4.8314916695930386,
            

In [6]:
def plot_arena(arena=None, spots=None, condition=None, ax=None):
    spot_colors = {'yeast': '#ffc04c', 'sucrose': '#4c8bff'}
    cond_colors = {'SAA': '#b6d7a8', 'AA': '#A4C2F4', 'S': '#EA9999', 'O': '#CCCCCC'}
    if ax is None:
        ax = plt.gca()
    ### artists
    if arena is not None:
        ax.set_xlim([-1.1*arena.ro, 1.1*arena.ro])
        ax.set_ylim([-1.1*arena.ro, 1.1*arena.ro])
        arena_border = plt.Circle((0, 0), arena.rr, color='k', fill=False)
        ax.add_artist(arena_border)
        outer_arena_border = plt.Circle((0, 0), arena.ro, color='#aaaaaa', fill=False)
        ax.add_artist(outer_arena_border)
        ax.plot(0, 0, 'o', color='black', markersize=2)
    if spots is not None:
        for each_spot in spots:
            substr = each_spot.substrate
            spot = plt.Circle((each_spot.rx, each_spot.ry), each_spot.rr, color=spot_colors[substr], alpha=0.5)
            ax.add_artist(spot)
    if condition is not None:
        if condition in cond_colors.keys():
            spot = plt.Rectangle((-arena.ro, arena.ro-2), 5, 5, color=cond_colors[condition])
            ax.add_artist(spot)
    ax.set_aspect("equal")
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.axis('off')
    return ax

def pdf_traj(df, scale, time=None, only=None, title=None):
    %matplotlib inline
    f, axes = plt.subplots(2, 2, figsize=(8,8), dpi=600)
    fly = 0
    for each_row in axes:
        for ax in each_row:
            arena = data.arenas[fly]
            if time is None:
                start, end = df.index[0], df.index[-1]
            else:
                start, end = time
            ax = plot_arena(arena=arena, spots=arena.spots, condition=data.condition[fly], ax=ax)
            x = df.loc[start:end, 'body_x']
            y = df.loc[start:end, 'body_y']
            hx = df.loc[start:end, 'head_x']
            hy = df.loc[start:end, 'head_y']
            tx = df.loc[start:end, 'tail_x']
            ty = df.loc[start:end, 'tail_y']
            if only is None or only == 'body':
                ax.plot(x/scale, y/scale, c='k')
            if only is None or only == 'tail':
                ax.scatter(tx/scale, ty/scale, c='b', s=.25)
            if only is None or only == 'head':
                ax.scatter(hx/scale, hy/scale, c='r', s=.25)
            fly += 1
    plt.tight_layout()
    f.savefig(title, dpi=600)